In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Load data into a pandas DataFrame
df = pd.read_csv("year_2013_2022.csv")
df['Infant'] = df[['< 1 yr',' 1 yr','2 yrs']].sum(axis=1)
df['Children'] = df.loc[:,'3 yrs':'15 yrs'].sum(axis=1)
df['Youth'] = df.loc[:,'16 yrs':'24 yrs'].sum(axis=1)
df['Adult'] = df.loc[:,'25 yrs':'59 yrs'].sum(axis=1)
df['Elderly'] = df.loc[:,'60 yrs':'100 yrs'].sum(axis=1) + df['> 100 yrs']
df.rename(columns={'ผลรวมประชากรทั้งหมด':'Total_Populations'},inplace=True)
AgeGroups = ['Infant','Children','Youth','Adult','Elderly']

df['Total_Populations'] = df['Total_Populations'].str.replace(',','')
df['Total_Populations'] = df['Total_Populations'].astype('float')

df.head()

,ปีเดือน,รหัสจังหวัด,ชื่อจังหวัด,รหัสสำนักทะเบียน,ชื่อสำนักทะเบียน,รหัสตำบล,ชื่อตำบล,รหัสหมู่บ้าน,ชื่อหมู่บ้าน,ชาย < 1 ปี,...,96 yrs,97 yrs,98 yrs,99 yrs,100 yrs,Infant,Children,Youth,Adult,Elderly
0,5612,0.0,ทั่วประเทศ,0.0,NaN,0.0,NaN,0.0,NaN,358688,...,9739,7917,6663,4674,4007,2255002,10444999,8615337,33601154,8734101
1,5612,10.0,กรุงเทพมหานคร,0.0,NaN,0.0,NaN,0.0,NaN,24126,...,923,817,645,551,445,154752,822094,715711,3007245,817853
2,5612,11.0,สมุทรปราการ,0.0,NaN,0.0,NaN,0.0,NaN,6976,...,196,150,118,119,75,43225,197426,157035,677341,147885
3,5612,12.0,นนทบุรี,0.0,NaN,0.0,NaN,0.0,NaN,5611,...,280,241,193,166,162,35615,168258,134619,636848,165484
4,5612,13.0,ปทุมธานี,0.0,NaN,0.0,NaN,0.0,NaN,5844,...,106,83,77,53,53,37209,170768,136969,579781,115878


In [2]:
total_pops = df[df['ชื่อจังหวัด']== 'ทั่วประเทศ'][['ชื่อจังหวัด','year','Total_Populations','Infant','Children','Youth','Adult','Elderly']].sort_values('year')

In [3]:
years = []
for i in range(2013,2051):
    years.append(i)
    # X_poly_pred = poly.fit_transform(X_pred)


In [4]:
import warnings
warnings.filterwarnings('ignore')

# total_pops['year'] = pd.to_datetime(total_pops['year'])
# total_pops.set_index('year', inplace=True)
ExpoSmoothPredTotal = pd.DataFrame()
# define the features and target
for Province in df['ชื่อจังหวัด'].drop_duplicates():
    ExpoSmoothPred = pd.DataFrame(years, columns = ["year"])
    total_pops = df[df['ชื่อจังหวัด']== Province][['ชื่อจังหวัด','year','Total_Populations','Infant','Children','Youth','Adult','Elderly']].sort_values('year')


    for AgeGroup in AgeGroups:
        y = total_pops[AgeGroup]
        # Set the date column as the index

        # Fit the model
        model = ExponentialSmoothing(y,trend='add').fit()

        # Make predictions
        predictions = model.predict(start=0, end=len(total_pops)+27)
        ExpoSmoothPred[AgeGroup] = predictions
        

    ExpoSmoothPred['Sum Prediction'] = ExpoSmoothPred.loc[:,"Infant":"Elderly"].sum(axis=1)
    #     ExpoSmoothPred.plot.line(x='year',y='Predicted '+AgeGroup+' Population')
    #     plt.show()
    y = total_pops['Total_Populations']
    # Set the date column as the index

    # Fit the model
    model = ExponentialSmoothing(y,trend='add').fit()

    # Make predictions
    predictions = model.predict(start=0, end=len(total_pops)+27)
    ExpoSmoothPred['Total_Populations'] = predictions
    ExpoSmoothPred.clip(lower=0, inplace = True)
    ExpoSmoothPred['Province'] = Province
    ExpoSmoothPredTotal = pd.concat([ExpoSmoothPredTotal,ExpoSmoothPred])

In [5]:
ExpoSmoothPredTotal.Province.nunique()

78

In [10]:
ExpoSmoothPredTotal

,year,Infant,Children,Youth,Adult,Elderly,Sum Prediction,Total_Populations,Province
0,2013,2.208311e+06,1.032836e+07,8.589490e+06,3.371744e+07,9.110754e+06,6.395435e+07,6.512472e+07,ทั่วประเทศ
1,2014,2.232291e+06,1.037452e+07,8.614950e+06,3.361112e+07,9.112600e+06,6.394548e+07,6.498266e+07,ทั่วประเทศ
2,2015,2.173518e+06,1.026494e+07,8.564153e+06,3.382402e+07,9.487379e+06,6.431401e+07,6.537933e+07,ทั่วประเทศ
3,2016,2.035985e+06,1.022741e+07,8.352827e+06,3.407065e+07,9.832547e+06,6.451942e+07,6.613108e+07,ทั่วประเทศ
4,2017,1.955335e+06,1.013073e+07,8.219950e+06,3.418837e+07,1.017884e+07,6.467323e+07,6.625162e+07,ทั่วประเทศ
...,...,...,...,...,...,...,...,...,...
33,2046,1.778941e+04,1.575873e+05,1.010553e+05,4.948427e+05,1.751880e+05,9.464627e+05,9.019286e+05,นราธิวาส
34,2047,1.708788e+04,1.568957e+05,1.003608e+05,4.994768e+05,1.783266e+05,9.521478e+05,9.055960e+05,นราธิวาส
35,2048,1.638635e+04,1.562041e+05,9.966636e+04,5.041110e+05,1.814651e+05,9.578330e+05,9.092634e+05,นราธิวาส
36,2049,1.568482e+04,1.555125e+05,9.897189e+04,5.087452e+05,1.846037e+05,9.635181e+05,9.129307e+05,นราธิวาส


In [11]:
ExpoSmoothPredTotal['DependencyRatio'] = ( ExpoSmoothPredTotal['Infant'] + ExpoSmoothPredTotal['Children']+ ExpoSmoothPredTotal['Elderly'])/(ExpoSmoothPredTotal['Youth'] + ExpoSmoothPredTotal['Adult'])  

In [12]:
ExpoSmoothPredTotal.head()

,year,Infant,Children,Youth,Adult,Elderly,Sum Prediction,Total_Populations,Province,DependencyRatio
0,2013,2.208311e+06,1.032836e+07,8.589490e+06,3.371744e+07,9.110754e+06,6.395435e+07,6.512472e+07,ทั่วประเทศ,0.511676
1,2014,2.232291e+06,1.037452e+07,8.614950e+06,3.361112e+07,9.112600e+06,6.394548e+07,6.498266e+07,ทั่วประเทศ,0.514360
2,2015,2.173518e+06,1.026494e+07,8.564153e+06,3.382402e+07,9.487379e+06,6.431401e+07,6.537933e+07,ทั่วประเทศ,0.517263
3,2016,2.035985e+06,1.022741e+07,8.352827e+06,3.407065e+07,9.832547e+06,6.451942e+07,6.613108e+07,ทั่วประเทศ,0.520842
4,2017,1.955335e+06,1.013073e+07,8.219950e+06,3.418837e+07,1.017884e+07,6.467323e+07,6.625162e+07,ทั่วประเทศ,0.525013


In [13]:
ExpoSmoothPredTotal.to_csv('ExpoSmoothPredTotal.csv',index=False)

In [14]:
reverse_pivot = ExpoSmoothPredTotal.melt(id_vars=['year','Province'], var_name='Age Gruop', value_name='Population')
reverse_pivot

,year,Province,Age Gruop,Population
0,2013,ทั่วประเทศ,Infant,2.208311e+06
1,2014,ทั่วประเทศ,Infant,2.232291e+06
2,2015,ทั่วประเทศ,Infant,2.173518e+06
3,2016,ทั่วประเทศ,Infant,2.035985e+06
4,2017,ทั่วประเทศ,Infant,1.955335e+06
...,...,...,...,...
23707,2046,นราธิวาส,DependencyRatio,5.882965e-01
23708,2047,นราธิวาส,DependencyRatio,5.873425e-01
23709,2048,นราธิวาส,DependencyRatio,5.864009e-01
23710,2049,นราธิวาส,DependencyRatio,5.854716e-01


In [9]:
reverse_pivot.to_csv('ExpoSmoothPredTotalRevPivot.csv',index=False)